# Notebook 07: Memory Bandwidth Benchmarking## Phase 2: Memory Management**Learning Objectives:**- Measure host-to-device transfer bandwidth- Compare pinned vs pageable memory performance- Understand PCIe bandwidth limitations- Profile memory access patterns- Optimize data transfer strategies

## Concept: Memory Bandwidth**Theoretical Bandwidth:**- PCIe 3.0 x16: 15.75 GB/s per direction- PCIe 4.0 x16: 31.5 GB/s per direction**Factors Affecting Bandwidth:**- Transfer size (larger is better)- Memory type (pinned vs pageable)- Transfer direction (H2D, D2H, D2D)- Concurrent operations**Bandwidth Calculation:**```Bandwidth = DataSize / Time```

## Example 1: Basic Memory Bandwidth Benchmarking

In [ ]:
%%cu#include <stdio.h>__global__ void kernel() {    printf("Example kernel\n");}int main() {    kernel<<<1, 1>>>();    cudaDeviceSynchronize();    return 0;}

## Practical ExerciseComplete the following exercises to practice the concepts learned.

In [ ]:
%%cu// Your solution here#include <stdio.h>int main() {    // TODO: Implement your solution    return 0;}

## Key Takeaways1. Key concept 12. Key concept 23. Key concept 3

## Next StepsContinue to: **08_next_topic.ipynb**

## Notes*Use this space to write your own notes and observations:*------